Downloading this data: https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires/data

### Load libraries

In [1]:
import os
import shutil

#Path where I saved kaggle.json 
source_path = '/Users/jonzimmerman/Desktop/Data Projects/Wildfires/data/kaggle.json' 

### Read in Kaggle token

In [2]:
#Destination path for kaggle.json
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)
shutil.copy(source_path, os.path.join(kaggle_dir, "kaggle.json"))

'/Users/jonzimmerman/.kaggle/kaggle.json'

In [3]:
#Set permission
os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

In [4]:
#!pip install kaggle

### Download SQL dataset

In [5]:
#!kaggle datasets download -d rtatman/188-million-us-wildfires

In [6]:
import zipfile

with zipfile.ZipFile("188-million-us-wildfires.zip", "r") as zip_ref:
    for file in zip_ref.namelist():
        if file.endswith(".sqlite"):
            zip_ref.extract(file, "wildfire_sqlite")
            print(f"Extracted {file} to wildfire_sqlite/")

Extracted FPA_FOD_20170508.sqlite to wildfire_sqlite/


### Just get the table you want

In [7]:
import sqlite3
import pandas as pd

db_path = "wildfire_sqlite/FPA_FOD_20170508.sqlite"
conn = sqlite3.connect(db_path)

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

#Make the pandas dataframe
df = pd.read_sql("SELECT * FROM Fires;", conn)

conn.close()

In [8]:
df.head(5)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [9]:
df.shape

(1880465, 39)

### Get rid of columns you don't want

In [22]:
df = df.drop(['OBJECTID', 'FOD_ID','SOURCE_SYSTEM_TYPE','SOURCE_SYSTEM','NWCG_REPORTING_AGENCY',
              'FPA_ID','NWCG_REPORTING_UNIT_ID','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME',
              'LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','ICS_209_INCIDENT_NUMBER',
               'ICS_209_NAME','MTBS_ID','MTBS_FIRE_NAME','COMPLEX_NAME','OWNER_CODE','OWNER_DESCR',
              'Shape'], axis=1)
df.head(3)

,NWCG_REPORTING_UNIT_NAME,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,Plumas National Forest,FOUNTAIN,2005,2453403.5,33,1300,9.0,Miscellaneous,2453403.5,33.0,1730,0.10,A,40.036944,-121.005833,CA,63,063,Plumas
1,Eldorado National Forest,PIGEON,2004,2453137.5,133,0845,1.0,Lightning,2453137.5,133.0,1530,0.25,A,38.933056,-120.404444,CA,61,061,Placer
2,Eldorado National Forest,SLACK,2004,2453156.5,152,1921,5.0,Debris Burning,2453156.5,152.0,2024,0.10,A,38.984167,-120.735556,CA,17,017,El Dorado


In [24]:
west_coast = df[df['STATE'].isin(['CA','OR','WA'])]

In [25]:
west_coast['STATE'].value_counts()

CA    189550
OR     61088
WA     33513
Name: STATE, dtype: int64

### Export so I can just reference the static data from Github

In [29]:
fire_data1 = west_coast[west_coast['STATE']=="WA"]
fire_data2 = west_coast[west_coast['STATE']=="OR"]
fire_data3 = west_coast[west_coast['STATE']=="CA"]
                        
print(fire_data1.shape)
print(fire_data2.shape)
print(fire_data3.shape)

(33513, 19)
(61088, 19)
(189550, 19)


In [30]:
fire_data1.to_csv('/Users/jonzimmerman/Desktop/Data Projects/Wildfires/data/fire_df_wa.csv',index=False)
fire_data2.to_csv('/Users/jonzimmerman/Desktop/Data Projects/Wildfires/data/fire_df_or.csv',index=False)
fire_data3.to_csv('/Users/jonzimmerman/Desktop/Data Projects/Wildfires/data/fire_df_ca.csv',index=False)